In [ ]:
from google.colab import drive
drive.mount._DEBUG = True
drive.mount('/content/drive', force_remount=True)

unset HISTFILE; export PS1="root@32ccfc405968-949703ee91e8405c9b57e49895241e2b: "
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/content# root@32ccfc405968-949703ee91e8405c9b57e49895241e2b: umount -f /content/drive || umount /content/drive; pkill -9 -x drive
root@32ccfc405968-949703ee91e8405c9b57e49895241e2b: pkill -9 -f /opt/google/drive/directoryprefetcher_binary
root@32ccfc405968-949703ee91e8405c9b57e49895241e2b: ( while `sleep 0.5`; do if [[ -d "/content/drive" && "$(ls -A /content/drive)" != "" ]]; then echo "google.colab.drive MOUNTED"; break; fi; done ) &
[1] 1410
root@32ccfc405968-949703ee91e8405c9b57e49895241e2b: cat /tmp/tmpl3g7d7l8/drive.fifo | head -1 | ( /opt/google/drive/drive --features=fuse_max_background:1000,max_read_qps:1000,max_write_qps:1000,max_operation_batch_size:15,max_parallel_push_task_instances:10,opendir_timeout_ms:120000,virtual_folders_omit_spaces:true --inet_family=IPV4_ONLY --metadata_ser

In [ ]:
import os
os.chdir('./drive/MyDrive/python_project/network')

In [ ]:
!pip install sentencepiece
# !git clone https://github.com/google-research/bert
# !pip install bert-for-tf2

In [ ]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import sentencepiece as spm

from glob import glob
# from google.colab import auth, drive
from tensorflow.keras.utils import Progbar

sys.path.append("bert")

from bert import modeling, optimization, tokenization
from bert.run_pretraining import input_fn_builder, model_fn_builder


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
from gensim.models.word2vec import Word2Vec
from gensim.models import FastText

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing import text, sequence
# from keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns
import os
import time

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
#     # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     strategy = tf.distribute.get_strategy() 
#     # default distribution strategy in Tensorflow. Works on CPU and single GPU.
# print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
network_df =pd.read_csv('./cic_payload.csv')
network_df.head()

,payload,label
0,00c1b114eb310023ae9b9567080045000034c2c8400040...,1
1,0023ae9b956700c1b114eb310800452805dc06bd400033...,1
2,0023ae9b956700c1b114eb31080045000061cea3400053...,1
3,0023ae9b956700c1b114eb310800450001e9cea4400053...,1
4,00c1b114eb310023ae9b9567080045000034c2c9400040...,1


In [ ]:
len(network_df)

159971

In [ ]:
network_df = network_df.drop_duplicates(['payload'])
len(network_df)

130944

In [ ]:
with open('./payload_0121.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(network_df['payload']))

In [ ]:
PRC_DATA_FPATH = "payload_0121.txt" 

In [ ]:
MODEL_PREFIX = "tokenizer_0121" #@param {type: "string"}
VOC_SIZE = 32000 #@param {type:"integer"}
SUBSAMPLE_SIZE = 12800000 #@param {type:"integer"}
NUM_PLACEHOLDERS = 256 #@param {type:"integer"}

SPM_COMMAND = ('--input={} --model_prefix={} '
               '--vocab_size={} --input_sentence_size={} '
               '--shuffle_input_sentence=true ' 
               '--bos_id=-1 --eos_id=-1').format(
               PRC_DATA_FPATH, MODEL_PREFIX, 
               VOC_SIZE - NUM_PLACEHOLDERS, SUBSAMPLE_SIZE)

# spm.SentencePieceTrainer.Train(SPM_COMMAND)

In [ ]:
def read_sentencepiece_vocab(filepath):
  voc = []
  with open(filepath, encoding='utf-8') as fi:
    for line in fi:
      voc.append(line.split("\t")[0])
  # skip the first <unk> token
  voc = voc[1:]
  return voc

snt_vocab = read_sentencepiece_vocab("{}.vocab".format(MODEL_PREFIX))
print("Learnt vocab size: {}".format(len(snt_vocab)))
print("Sample tokens: {}".format(random.sample(snt_vocab, 10)))

Learnt vocab size: 31743
Sample tokens: ['31373035323', '818197', '66434386', '39000004', '20202036373', '3080045000034993', '365937', '6433317', '6676869', '5362']


In [ ]:
def parse_sentencepiece_token(token):
    if token.startswith("▁"):
        return token[1:]
    else:
        return "##" + token

bert_vocab = list(map(parse_sentencepiece_token, snt_vocab))
ctrl_symbols = ["[PAD]","[UNK]","[CLS]","[SEP]","[MASK]"]
bert_vocab = ctrl_symbols + bert_vocab

In [ ]:
bert_vocab += ["[UNUSED_{}]".format(i) for i in range(VOC_SIZE - len(bert_vocab))]
print(len(bert_vocab))

32000


In [ ]:
VOC_FNAME = "vocab.txt" #@param {type:"string"}

with open(VOC_FNAME, "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [ ]:
MAX_SEQ_LENGTH = 512 #@param {type:"integer"}
MASKED_LM_PROB = 0.15 #@param
MAX_PREDICTIONS = 20 #@param {type:"integer"}
DO_LOWER_CASE = False #@param {type:"boolean"}
PROCESSES = 4 #@param {type:"integer"}
PRETRAINING_DIR = "pretraining_data" #@param {type:"string"}


In [ ]:
# !mkdir ./shards
# !split -a 4 -l 256000 -d $PRC_DATA_FPATH ./shards/shard_
# !ls ./shards/

In [ ]:
XARGS_CMD = ("ls ./shards/ | "
             "xargs -n 1 -P {} -I{} "
             "python3 bert/create_pretraining_data.py "
             "--input_file=./shards/{} "
             "--output_file={}/{}.tfrecord "
             "--vocab_file={} "
             "--do_lower_case={} "
             "--max_predictions_per_seq={} "
             "--max_seq_length={} "
             "--masked_lm_prob={} "
             "--random_seed=34 "
             "--dupe_factor=5")

XARGS_CMD = XARGS_CMD.format(PROCESSES, '{}', '{}', PRETRAINING_DIR, '{}', 
                            VOC_FNAME, DO_LOWER_CASE, 
                            MAX_PREDICTIONS, MAX_SEQ_LENGTH, MASKED_LM_PROB)

tf.io.gfile.makedirs(PRETRAINING_DIR)
# !$XARGS_CMD//

In [ ]:
BUCKET_NAME = "network" #@param {type:"string"}
MODEL_DIR = "bert_model" #@param {type:"string"}
tf.io.gfile.makedirs(MODEL_DIR)

In [ ]:
bert_base_config = {
  "attention_probs_dropout_prob": 0.1, 
  "directionality": "bidi", 
  "hidden_act": "gelu", 
  "hidden_dropout_prob": 0.1, 
  "hidden_size": 768, 
  "initializer_range": 0.02, 
  "intermediate_size": 3072, 
  "max_position_embeddings": 512, 
  "num_attention_heads": 12, 
  "num_hidden_layers": 12, 
  "pooler_fc_size": 768, 
  "pooler_num_attention_heads": 12, 
  "pooler_num_fc_layers": 3, 
  "pooler_size_per_head": 128, 
  "pooler_type": "first_token_transform", 
  "type_vocab_size": 2, 
  "vocab_size": VOC_SIZE
}

with open("{}/bert_config.json".format(MODEL_DIR), "w") as fo:
  json.dump(bert_base_config, fo, indent=2)
  
with open("{}/{}".format(MODEL_DIR, VOC_FNAME), "w") as fo:
  for token in bert_vocab:
    fo.write(token+"\n")

In [ ]:
!python bert/run_pretraining.py \
--input_file=pretraining_data/shard_0000.tfrecord \
--output_dir=bert_model/ \
--do_train=True \
--do_eval=True \
--bert_config_file=bert_model/bert_config.json \
--train_batch_size=4 \
--max_seq_length=512 \
--max_predictions_per_seq=20 \
--num_warmup_steps=1000 \
--num_train_steps=30000 \
--learning_rate=1e-4 \
--save_checkpoints_steps=1000 \
--do_lower_case=False

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
INFO:tensorflow:global_step/sec: 6.42302
I0124 06:19:00.818302 140426394118016 tpu_estimator.py:2387] global_step/sec: 6.42302
INFO:tensorflow:examples/sec: 25.6921
I0124 06:19:00.818614 140426394118016 tpu_estimator.py:2388] examples/sec: 25.6921
INFO:tensorflow:global_step/sec: 6.42618
I0124 06:19:01.129526 140426394118016 tpu_estimator.py:2387] global_step/sec: 6.42618
INFO:tensorflow:examples/sec: 25.7047
I0124 06:19:01.129839 140426394118016 tpu_estimator.py:2388] examples/sec: 25.7047
INFO:tensorflow:global_step/sec: 6.41369
I0124 06:19:01.441374 140426394118016 tpu_estimator.py:2387] global_step/sec: 6.41369
INFO:tensorflow:examples/sec: 25.6548
I0124 06:19:01.441700 140426394118016 tpu_estimator.py:2388] examples/sec: 25.6548
INFO:tensorflow:global_step/sec: 6.43151
I0124 06:19:01.752339 140426394118016 tpu_estimator.py:2387] global_step/sec: 6.43151
INFO:tensorflow:examples/sec: 25.726
I0124 06:19:01.752552 140426394118016 tpu_estimator.py:2

In [ ]:
network_df['id'] = network_df.index
print(network_df)

                                                  payload  label      id
0       00c1b114eb310023ae9b9567080045000034c2c8400040...      1       0
1       0023ae9b956700c1b114eb310800452805dc06bd400033...      1       1
2       0023ae9b956700c1b114eb31080045000061cea3400053...      1       2
3       0023ae9b956700c1b114eb310800450001e9cea4400053...      1       3
4       00c1b114eb310023ae9b9567080045000034c2c9400040...      1       4
...                                                   ...    ...     ...
159966  00c1b114eb310023ae9badb308004500003427b0400040...      0  159966
159967  00c1b114eb310023ae9badb30800450001700bd2400040...      0  159967
159968  0023ae9badb300c1b114eb3108004500025e9bd2400031...      0  159968
159969  00c1b114eb310023ae9badb308004500019d0bd3400040...      0  159969
159970  00c1b114eb31b8ac6f3607ee08004500002942d5400080...      0  159970

[130944 rows x 3 columns]


In [ ]:
network_df = network_df[['id', 'payload', 'label']]
network_df

,id,payload,label
0,0,00c1b114eb310023ae9b9567080045000034c2c8400040...,1
1,1,0023ae9b956700c1b114eb310800452805dc06bd400033...,1
2,2,0023ae9b956700c1b114eb31080045000061cea3400053...,1
3,3,0023ae9b956700c1b114eb310800450001e9cea4400053...,1
4,4,00c1b114eb310023ae9b9567080045000034c2c9400040...,1
...,...,...,...
159966,159966,00c1b114eb310023ae9badb308004500003427b0400040...,0
159967,159967,00c1b114eb310023ae9badb30800450001700bd2400040...,0
159968,159968,0023ae9badb300c1b114eb3108004500025e9bd2400031...,0
159969,159969,00c1b114eb310023ae9badb308004500019d0bd3400040...,0


In [ ]:
network_df.columns=['id', 'document', 'label']

In [ ]:
network_df

,id,document,label
0,0,00c1b114eb310023ae9b9567080045000034c2c8400040...,1
1,1,0023ae9b956700c1b114eb310800452805dc06bd400033...,1
2,2,0023ae9b956700c1b114eb31080045000061cea3400053...,1
3,3,0023ae9b956700c1b114eb310800450001e9cea4400053...,1
4,4,00c1b114eb310023ae9b9567080045000034c2c9400040...,1
...,...,...,...
159966,159966,00c1b114eb310023ae9badb308004500003427b0400040...,0
159967,159967,00c1b114eb310023ae9badb30800450001700bd2400040...,0
159968,159968,0023ae9badb300c1b114eb3108004500025e9bd2400031...,0
159969,159969,00c1b114eb310023ae9badb308004500019d0bd3400040...,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(network_df['document'], network_df['label'], test_size=0.2, random_state=4)
print(X_train.shape, X_test.shape)
# print(type(X_train), type(X_test))
# print(list(X_train[:3]))
# exit()

(104755,) (26189,)


In [ ]:
y_train = y_train.reset_index()
y_train

,index,label
0,96620,0
1,75466,0
2,115133,0
3,42082,1
4,123155,0
...,...,...
104750,148571,0
104751,140702,0
104752,114162,0
104753,93489,0


In [ ]:
X_train = X_train.reset_index()
X_train

,index,document
0,96620,b8ac6f3607ee00c1b114eb31080045000496e914400039...
1,75466,b8ac6f1d1f6c1866da9be37d0800450000ea5e01000080...
2,115133,00c1b114eb310023ae9b8abf08004500004890a0400040...
3,42082,b8ac6f3604e300c1b114eb310800452805dc4ef2400033...
4,123155,b8ac6f3604e300c1b114eb310800450005dcd1a84000f2...
...,...,...
104750,148571,00c1b114eb31b8ac6f1d1f6c08004500002816cd400080...
104751,140702,00c1b114eb31b8ac6f3607ee08004500002970bf400080...
104752,114162,0023ae9b8abf00c1b114eb31080045080b8481d9400035...
104753,93489,b8ac6f1d1f6c00c1b114eb310800450005dce91a4000eb...


In [ ]:
X_train['label'] = y_train['label']
X_train

,index,document,label
0,96620,b8ac6f3607ee00c1b114eb31080045000496e914400039...,0
1,75466,b8ac6f1d1f6c1866da9be37d0800450000ea5e01000080...,0
2,115133,00c1b114eb310023ae9b8abf08004500004890a0400040...,0
3,42082,b8ac6f3604e300c1b114eb310800452805dc4ef2400033...,1
4,123155,b8ac6f3604e300c1b114eb310800450005dcd1a84000f2...,0
...,...,...,...
104750,148571,00c1b114eb31b8ac6f1d1f6c08004500002816cd400080...,0
104751,140702,00c1b114eb31b8ac6f3607ee08004500002970bf400080...,0
104752,114162,0023ae9b8abf00c1b114eb31080045080b8481d9400035...,0
104753,93489,b8ac6f1d1f6c00c1b114eb310800450005dce91a4000eb...,0


In [ ]:
# 인덱스 없애기
X_train.to_csv('data/train.tsv', sep = '\t', index = False)

In [ ]:
y_test = y_test.reset_index()
X_test = X_test.reset_index()
X_test['label'] = y_test['label']
X_test

,index,document,label
0,141480,00c1b114eb31b8ac6f3607ee080045000028688d400080...,0
1,92640,0023ae9b8abf00c1b114eb310800450005dc3450400034...,0
2,23557,1866da9be37d0023ae9b8abf0800450000480a9f400040...,1
3,25293,0023ae9b95671866da9be37d0800450000745bb2000080...,1
4,99374,1866da9be37db8ac6f3607ee0800450000426d27000080...,0
...,...,...,...
26184,70280,b8ac6f3604e300c1b114eb310800450005dc50964000f1...,0
26185,89213,b8ac6f1d1f6c00c1b114eb31080045000056db3a000034...,0
26186,61373,00c1b114eb310019b90a69f10800450000348d1b400040...,0
26187,111708,0023ae9b8abf00c1b114eb310800450005be524d000034...,0


In [ ]:
# 인덱스 없애기
X_test.to_csv('data/dev.tsv', sep = '\t', index = False)

In [ ]:
!python bert/run_classifier.py \
--task_name=nsmc \
--do_train=true \
--do_eval=true \
--data_dir=data \
--vocab_file=vocab.txt \
--bert_config_file=bert_model/bert_config.json \
--init_checkpoint=bert_model/model.ckpt-30002 \
--max_seq_length=512 \
--train_batch_size=4 \
--eval_batch_size=4 \
--num_train_epochs=10.0 \
--learning_rate=3e-5 \
--do_lower_case=True \
--output_dir=payload_output

Instructions for updating:
non-resource variables are not supported in the long term

W0124 14:13:35.483076 140059077486464 module_wrapper.py:155] From bert/run_classifier.py:885: The name tf.estimator.tpu.InputPipelineConfig is deprecated. Please use tf.compat.v1.estimator.tpu.InputPipelineConfig instead.

W0124 14:13:38.065693 140059077486464 estimator.py:2031] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f615008fef0>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'payload_output', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_dis